In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [6]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs = BeautifulSoup(page,'xml')

In [7]:
table = bs.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [8]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [9]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[(~df['PostalCode'].isnull()) & (df['Borough'] != 'Not assigned')]
df = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(list)
df = pd.DataFrame(df)
df1 = df.sort_values(by=['Neighbourhood'])
df1 = df1.reset_index()
df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M5H,Downtown Toronto,"[Adelaide, King, Richmond]"
1,M1S,Scarborough,[Agincourt]
2,M1V,Scarborough,"[Agincourt North, L'Amoreaux East, Milliken, S..."
3,M9V,Etobicoke,"[Albion Gardens, Beaumond Heights, Humbergate,..."
4,M8W,Etobicoke,"[Alderwood, Long Branch]"
5,M3H,North York,"[Bathurst Manor, Downsview North, Wilson Heights]"
6,M2K,North York,[Bayview Village]
7,M5M,North York,"[Bedford Park, Lawrence Manor East]"
8,M5E,Downtown Toronto,[Berczy Park]
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


#### In this notebook, I used Beatifullsoup library to scrap the Wikipedia page. Then, I transforme the data in format requiered, cleaning data remove the Borough whit Not assigned and missings

In [107]:
df1.shape

(103, 3)

In [ ]:
#import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario') #.format(**geodata))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [12]:
path ='http://cocl.us/Geospatial_data'
dfl = pd.read_csv(path)
dfl.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
dfc = pd.merge(df1, dfl, left_on ='PostalCode', right_on = 'Postal Code')
dfc.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M5H,Downtown Toronto,"[Adelaide, King, Richmond]",M5H,43.650571,-79.384568
1,M1S,Scarborough,[Agincourt],M1S,43.794200,-79.262029
2,M1V,Scarborough,"[Agincourt North, L'Amoreaux East, Milliken, S...",M1V,43.815252,-79.284577
3,M9V,Etobicoke,"[Albion Gardens, Beaumond Heights, Humbergate,...",M9V,43.739416,-79.588437
4,M8W,Etobicoke,"[Alderwood, Long Branch]",M8W,43.602414,-79.543484


In [ ]:
=